In [2]:
import requests
import pandas as pd
import numpy as np
import json

In [3]:
#testing
r = requests.post("https://japerk-text-processing.p.mashape.com/sentiment/", headers={
    "X-Mashape-Key": "",
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
  }, data={'text': "awful awful awful awful"})

print(r.text)

{"probability": {"neg": 0.82807138349387666, "neutral": 0.09569942540264019, "pos": 0.1719286165061234}, "label": "neg"}


In [4]:
#load sample data from local file
data = pd.read_csv("file://localhost/C:/Users/ajamar/Desktop/output_2011_abridged.csv")

In [10]:
#change index name, add columns for the different sentiment analysis tools
data.index.rename("Row Number",inplace=True)
data["japerk"] = ""
data["textanalysis"] = ""
data["sentimentapi"] = ""

In [11]:
def sentiment_analysis_japerk(df,tweets):
    """Sentiment Analysis Tool #1"""
    ratings = []
    counter = 0
    for tweet in tweets:
        api_request = requests.post("https://japerk-text-processing.p.mashape.com/sentiment/", headers={
    "X-Mashape-Key": "",
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
  }, data={'text': tweet})
        rating_dict = json.loads(api_request.text)
        rating = rating_dict["label"]
        if rating == "neutral":
            ratings.append(0)
            df.ix[counter, "japerk"] = 0
        elif rating == "pos":
            ratings.append(1)
            df.ix[counter, "japerk"] = 1
        elif rating == "neg":
            ratings.append(-1)
            df.ix[counter, "japerk"] = -1
        counter += 1
    return np.mean(ratings)

In [12]:
def sentiment_analysis_textanalysis(df,tweets):
    """Sentiment Analysis Tool #2"""    
    ratings = []
    counter = 0
    for tweet in tweets:
        api_request = requests.post("https://textanalysis-text-sentiment-v1.p.mashape.com/sentiment-analyzer", headers={
    "X-Mashape-Key": "",
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
  }, data={'text': tweet})
        rating_dict = json.loads(api_request.text)
        rating = rating_dict["sentiment"]
        if rating == "neutral":
            ratings.append(0)
            df.ix[counter, "textanalysis"] = 0
        elif rating == "positive":
            ratings.append(1)
            df.ix[counter, "textanalysis"] = 1
        elif rating == "negative":
            ratings.append(-1)
            df.ix[counter, "textanalysis"] = -1
        counter += 1
    return np.mean(ratings)

In [13]:
def sentiment_analysis_sentimentapi(df,tweets):
    """Sentiment Analysis Tool #3"""
    ratings = []
    counter = 0
    for tweet in tweets:
        api_request = requests.post("https://sentimentapi.p.mashape.com/products/api/extractsentiment?", headers={
    "X-Mashape-Key": "",
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
  }, data={'text': tweet})
        rating_dict = json.loads(api_request.text)
        rating = float(rating_dict["rating"])
        ratings.append(rating)
        df.ix[counter, "sentimentapi"] = rating
        counter += 1
    return np.mean(ratings)

In [14]:
data.head()

,Date,N/A,Unknown,Tweet,Blank,Tags,Hashtags,Not Applicable,URL,japerk,textanalysis,sentimentapi
Row Number,,,,,,,,,,,,
0,1/3/2011 23:59,1,1,Bringing a repeal of Obamacare to a vote on th...,NaN,NaN,NaN,2.210000e+16,https://twitter.com/illinoispolicy/status/2207...,,,
1,1/3/2011 23:59,0,0,#Obamacare working out great so far! My family...,NaN,NaN,#Obamacare #Blue,2.210000e+16,https://twitter.com/edfranks/status/2207954535...,,,
2,1/3/2011 23:58,0,0,MT @KevinMaddenDC GOP frames HCR repeal as job...,NaN,@KevinMaddenDC,NaN,2.210000e+16,https://twitter.com/RebeccaCarelli/status/2207...,,,
3,1/3/2011 23:57,0,0,With new Congress being sworn in this week can...,NaN,NaN,NaN,2.210000e+16,https://twitter.com/bfuller96/status/220792523...,,,
4,1/3/2011 23:56,0,0,@johnboehner please vote for a repel of Obamac...,NaN,@johnboehner,NaN,2.210000e+16,https://twitter.com/FredPond/status/2207898792...,,,


In [16]:
data = data.head()

In [17]:
sentiment_analysis_japerk(data, data["Tweet"])

0.0

In [33]:
#read in larger set of data from local file
full_data = pd.read_csv("file://localhost/C:/Users/ajamar/Desktop/output_2011_5000.csv")

In [34]:
#change index name, add columns for the different sentiment analysis tools
full_data.index.rename("Row Number",inplace=True)
full_data["japerk"] = ""
full_data["textanalysis"] = ""
full_data["sentimentapi"] = ""

In [36]:
#limit data to 1,000 rows
full_data = full_data.head(1000)

In [37]:
#test sentiment analysis package 1
sentiment_analysis_japerk(full_data, full_data["Tweet"])

0.13200000000000001

In [38]:
#test sentiment analysis package 2
sentiment_analysis_textanalysis(full_data, full_data["Tweet"])

-0.57799999999999996

In [39]:
#test sentiment analysis package 3
sentiment_analysis_sentimentapi(full_data, full_data["Tweet"])

3.7360000000000002

In [45]:
#save results to csv
full_data.to_csv("205_sentiment_analysis", encoding = 'utf-8')